## Processo de Atualização de Dados Automatizada (TOTVS ➞ Google Sheets)

In [ ]:
# DADOS IMPORTANTES PARA SEREM INICIALIZADOS.
# ATENÇÃO: MUDAR MÊS NO 2º DIA DO MÊS
# ATENÇÃO: MUDAR SEMANA SEMPRE NA TERÇA
# ATENÇÃO: MUDAR LINHA DO TME DIÁRIO TODOS OS DIAS (variável: TMESegundaTabela)

semana = 'W10' #Somar mais um na semana todas as terças (Ex.: W10, W11, W12, W13, ...)
mes = 'Mar' #Mudar mes no segundo dia do mês (Ex.: Mar, Apr, May, Jun, Jul, ...)
mesAno = 'Mar 2024' #Mudar mesAno no segundo dia do mês (Ex.: Mar, Apr, May, Jun, Jul, ...)


#Linhas que serão atualizadas:

# - Guia DB_Pacientes_Actual:
#Linhas da primeira e segunda tabelas da aba (Total)
totalPrimeiraTabela = 'DB_Pacientes_Actual!B40' #Somar mais um na linha no segundo dia do mês
totalSegundaTabela = 'DB_Pacientes_Actual!F21' #Somar mais um na linha no segundo dia do mês
#Linhas da primeira e segunda tabelas da aba (Exame)
examePrimeiraTabela = 'DB_Pacientes_Actual!M18' #Somar mais um na linha no segundo dia do mês
exameSegundaTabela = 'DB_Pacientes_Actual!Q21' #Somar mais um na linha no segundo dia do mês
#Linhas da primeira e segunda tabelas da aba (Consulta)
consultaPrimeiraTabela = 'DB_Pacientes_Actual!X18' #Somar mais um na linha no segundo dia do mês
consultaSegundaTabela = 'DB_Pacientes_Actual!AB21' #Somar mais um na linha no segundo dia do mês
#Linhas da primeira e segunda tabelas da aba (Odonto)
odontoPrimeiraTabela = 'DB_Pacientes_Actual!AI18' #Somar mais um na linha no segundo dia do mês
odontoSegundaTabela = 'DB_Pacientes_Actual!AM21' #Somar mais um na linha no segundo dia do mês

# - Guia DB_TME_Actual:
#Linhas referente ao mês (DEVE SER ATUALIZADA MENSALMENTE A PARTIR DO DIA 02)
TMEPrimeiraTabelaC1 = 'DB_TME_Actual!B22' #Somar mais um na linha no segundo dia do mês
TMEPrimeiraTabelaC2 = 'DB_TME_Actual!C22' #Somar mais um na linha no segundo dia do mês
TMEPrimeiraTabelaC3 = 'DB_TME_Actual!D22' #Somar mais um na linha no segundo dia do mês
#linha referente ao dia (DEVE SER ATUALIZADA DIARIAMENTE)
TMESegundaTabela = 'DB_TME_Actual!H139' #Somar mais um na linha TODOS OS DIAS
#linha referente a semana (DEVE SER MUDADO NA TERÇA)
TMETerceiraTabela = 'DB_TME_Actual!N25' #Somar mais um na linha nas terças e todas as semanas

In [ ]:
#Verifica se as bibliotecas estão instaladas, se não instala-as.
import importlib

biblioteca = ['imaplib', 'datetime', 'email', 'os', 'json', 'numpy', 'pandas', 'gspread']#, 'google-api-python-client', 'google-auth-httplib2', 'google-auth-oauthlib']

for i in biblioteca:
    try:
        importlib.import_module(i)
        print(f"A biblioteca {i} já está instalada.")
    except ImportError:
        print(f"A biblioteca {i} não está instalada. Instalando agora...")
        !pip install {i}
        print(f"A biblioteca {i} foi instalada com sucesso.")

In [ ]:
#Criação das pastas
import os

pastas = ['saída', 'anexos']

for i in pastas:
    if not os.path.exists(i):
        os.makedirs(i)
        print(f"Pasta '{i}' criada com sucesso!")
    else:
        print(f"A pasta '{i}' já existe.")

In [ ]:
#Apaga todos os arquivos baixados do e-mail
import os

pasta = 'saída'
arquivos = os.listdir(pasta)
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    if os.path.isfile(caminho_arquivo):
        os.remove(caminho_arquivo)
print(f"Todos os arquivos em -{pasta}- foram apagados.")

In [ ]:
#Acessa e-mail e pega anexos de um email específico
from datetime import datetime, date
import imaplib
import email
import os
import json

with open('credenciais.json') as f:
    config = json.load(f)

EMAIL_CONFIG = config['email']
IMAP_SERVER = EMAIL_CONFIG['imap_server']
EMAIL = EMAIL_CONFIG['email_address']
PASSWORD = EMAIL_CONFIG['password']
TARGET_EMAIL_SUBJECT = EMAIL_CONFIG['target_email_subject']
DOWNLOAD_FOLDER = 'anexos'

today = date.today().strftime('%d-%b-%Y')

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL, PASSWORD)
mail.select('inbox')

status, data = mail.search(None, f'(SUBJECT "{TARGET_EMAIL_SUBJECT}")')

for num in data[0].split():
    status, msg_data = mail.fetch(num, '(RFC822)')
    raw_email = msg_data[0][1]
    msg = email.message_from_bytes(raw_email)
    
    email_date = msg['Date']
    email_date = datetime.strptime(email_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%d-%b-%Y')
    
    if email_date == today:
        if msg.get_content_maintype() == 'multipart':
            for part in msg.walk():
                if part.get_content_maintype() == 'multipart' or part.get('Content-Disposition') is None:
                    continue

                filename = part.get_filename()
                if filename:
                    filepath = os.path.join(DOWNLOAD_FOLDER, filename)
                    if not os.path.isfile(filepath):
                        with open(filepath, 'wb') as f:
                            f.write(part.get_payload(decode=True))

mail.close()
mail.logout()

print("Anexos do e-mail baixado!")

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

#Criação do Dataframe Semanas|Início da Semana
n_semanas = 100
dados_semanas = []
inicio_semana = datetime(2024, 1, 1)
for i in range(1, n_semanas + 1):
    semana = f'W{i}'
    dados_semanas.append((semana, inicio_semana.strftime('%d/%m/%Y')))
    inicio_semana += timedelta(days=7)
df_semanas = pd.DataFrame(dados_semanas, columns=['Semana', 'Início da Semana'])

#Acessa pasta trata como Dataframe + algumas mudanças
pasta = 'anexos'
dfs = []
dfs1 = [] 
dfs2 = []
dfs3 = []

for arquivo in os.listdir(pasta):
    if arquivo.startswith('Pacientes_Month') or arquivo.startswith('Pacientes_Month_exames') or arquivo.startswith('Pacientes_Month_Consultas') or arquivo.startswith('Pacientes_Month_Odonto') or arquivo.startswith('GSV') or arquivo.startswith('Planning') or arquivo.startswith('Avaliações'):
        caminho_arquivo = os.path.join(pasta, arquivo)
        df = pd.read_excel(caminho_arquivo)
        dfs.append(df)

for arquivo in os.listdir(pasta):
    if arquivo.startswith('Pacientes_Month_Unidades') or arquivo.startswith('Pacientes_Month_exames_unidades') or arquivo.startswith('Pacientes_Month_Consultas_unidades') or arquivo.startswith('Pacientes_Month_Odonto_unidades'):
        caminho_arquivo = os.path.join(pasta, arquivo)
        df = pd.read_excel(caminho_arquivo)
        df['Month/Year (Data_EXECUCAO)'] = pd.to_datetime(df['Month/Year (Data_EXECUCAO)'], format='%b %Y').dt.strftime('%m/%d/%Y')
        dfs1.append(df)
        
for arquivo in os.listdir(pasta):
    if arquivo.startswith('TME'):
        caminho_arquivo = os.path.join(pasta, arquivo)
        df = pd.read_excel(caminho_arquivo, usecols=lambda col: col not in {'% SLA Recepção', 'TMA Consulta', '% SLA Consulta'})
        dfs2.append(df)

for arquivo in os.listdir(pasta):
    if arquivo.startswith('Pacientes_Week_Consulta') or arquivo.startswith('Pacientes_Week_exames') or arquivo.startswith('Pacientes_Week_Consultas') or arquivo.startswith('Pacientes_Week_Odonto'):
        caminho_arquivo = os.path.join(pasta, arquivo)
        df = pd.read_excel(caminho_arquivo)
        df = df.rename(columns={'Week (Mon-Sun) (Data_EXECUCAO)': 'Semana'})
        df = df.merge(df_semanas, how='left', on='Semana')
        dfs3.append(df)
        

print("Anexos tratados e levantados como Dataframe!")

In [ ]:
#DESCRIÇÃO DOS DATAFRAMES

#dfs[0] = Avaliações
#dfs[1] = GSV
#dfs[2] = Pac Pagantes 2º tabela
#dfs[3] = Consultas 2º tabela
#dfs[4] = Consultas 3ª tabela
#dfs[5] = Exames 2ª tabela
#dfs[6] = Exames 3ª tabela
#dfs[7] = Odonto 2ª Tabela
#dfs[8] = Odonto 3ª tabela
#dfs[9] = Pac Pagantes 3ª tabela
#dfs[10] = Orçamentos

#dfs1[0] = Consultas 3º tabela
#dfs1[1] = Exames 3º tabela
#dfs1[2] = Odonto 3º tabela
#dfs1[3] = Pac Pagantes 3º tabela

#dfs2[0] = TMA 1º coluna
#dfs2[1] = TMA 2ª coluna
#dfs2[2] = TMA 3º coluna

#dfs3[0] = Pac Pagantes 1º tabela
#dfs3[1] = Consultas 1º tabela
#dfs3[2] = Exames 1º tabela
#dfs3[3] = Odonto 1º tabela

In [ ]:
#Pac Pagantes
df_pac_pagantes = pd.concat([dfs3[0], pd.DataFrame(columns=[' ']), dfs[2], pd.DataFrame(columns=[' ']), dfs1[3]], axis=1)
df_pac_pagantes.to_excel('saída/pac_pagantes.xlsx', index=False)

In [ ]:
#Exames
df_exames = pd.concat([dfs3[2], pd.DataFrame(columns=[' ']), dfs[5], pd.DataFrame(columns=[' ']), dfs1[1]], axis=1)
df_exames.to_excel('saída/exames.xlsx', index=False)

In [ ]:
#Consultas
df_consultas = pd.concat([dfs3[1], pd.DataFrame(columns=[' ']), dfs[3], pd.DataFrame(columns=[' ']), dfs1[0]], axis=1)
df_consultas.to_excel('saída/consultas.xlsx', index=False)

In [ ]:
#Odonto
df_odonto = pd.concat([dfs3[3], pd.DataFrame(columns=[' ']), dfs[7], pd.DataFrame(columns=[' ']), dfs1[2]], axis=1)
df_odonto.to_excel('saída/odonto.xlsx', index=False)

In [ ]:
#GSV
dfs[1]['Date (Data_EXECUCAO)'] = pd.to_datetime(dfs[1]['Date (Data_EXECUCAO)'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y')
dfs[1].to_excel('saída/GSV.xlsx', index=False)

In [ ]:
#TMA
import numpy as np
dfs2[0][['TME Senha', 'TMA Recepção', 'TME Consulta']] = np.ceil(dfs2[0][['TME Senha', 'TMA Recepção', 'TME Consulta']])
dfs2[1][['TME Senha', 'TMA Recepção', 'TME Consulta']] = np.ceil(dfs2[1][['TME Senha', 'TMA Recepção', 'TME Consulta']])
dfs2[2][['TME Senha', 'TMA Recepção', 'TME Consulta']] = np.ceil(dfs2[2][['TME Senha', 'TMA Recepção', 'TME Consulta']])
#dfs2[1]['Date (Data_EXECUCAO)'] = pd.to_datetime(dfs2[1]['Date (Data_EXECUCAO)'], format='%m/%d/%Y').dt.strftime('%m/%d/%Y')
df_TMA = pd.concat([dfs2[0], pd.DataFrame(columns=[' ']), dfs2[1], pd.DataFrame(columns=[' ']), dfs2[2]], axis=1)
df_TMA.to_excel('saída/TMA.xlsx', index=False)

In [ ]:
#Orçamentos
dfs[10]['DATA'] = pd.to_datetime(dfs[10]['DATA'], format='%m/%d/%Y').dt.strftime('%m/%d/%Y')
dfs[10].to_excel('saída/orçamentos.xlsx', index=False)

In [ ]:
#Avaliações
dfs[0]['Date (Data_EXECUCAO)'] = pd.to_datetime(dfs[0]['Date (Data_EXECUCAO)'], format='%m/%d/%Y').dt.strftime('%m/%d/%Y')
dfs[0].to_excel('saída/avaliações.xlsx', index=False)

print("Bases tratadas e salvas em arquivos Excel!")

In [ ]:
#Apaga todos os arquivos baixados do e-mail
import os

pasta = 'anexos'
arquivos = os.listdir(pasta)
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    if os.path.isfile(caminho_arquivo):
        os.remove(caminho_arquivo)
print(f"Todos os arquivos em -{pasta}- foram apagados.")

In [ ]:
#Autenticação e autorização 
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Escopos: se será somente leitura, escrita ou ambos
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

#Autenticação e Autorização
creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file("token_sheets.json", SCOPES)
      creds = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
      token.write(creds.to_json())
    
print("Autenticação e autorização efetuada com sucesso!")

In [ ]:
#Início das edições da planilha no Google Sheets
# Identificação (ID) da planilha
SAMPLE_SPREADSHEET_ID = "DIGITE AQUI A ID DA PLANILHA DO GOOGLE SHEETS"
    
#Acesso e edição da planilha
try:
    service = build("sheets", "v4", credentials=creds)
    sheet = service.spreadsheets()
    
    #Edição guia "DB_Pacientes_Actual":
    
     #1ª tabela TOTAL - ESPECIFICAR SEMANA
    df_filtrado = dfs3[0].loc[df.eq(semana).any(axis=1)]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = totalPrimeiraTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute())
     #2ª tabela TOTAL - ESPECIFICAR MÊS
    df_filtrado = dfs[2][dfs[2]['Month (Data_EXECUCAO)'] == mes]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = totalSegundaTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute()) 
     #3ª tabela TOTAL
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = 'DB_Pacientes_Actual!H4', valueInputOption = 'USER_ENTERED', body = {'values': dfs[9].values.tolist()}).execute())
    #Apaga as linhas abaixo dos dados adicionados
    numero_linhas_dados = len(dfs[9])
    intervalo_apagar = f'DB_Pacientes_Actual!H{numero_linhas_dados + 100}:J'
    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=intervalo_apagar).execute()
    
     #1ª tabela EXAME - ESPECIFICAR SEMANA
    df_filtrado = dfs3[2].loc[df.eq(semana).any(axis=1)]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = examePrimeiraTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute())
     #2ª tabela EXAME - ESPECIFICAR MÊS
    df_filtrado = dfs[5][dfs[5]['Month (Data_EXECUCAO)'] == mes]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = exameSegundaTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute()) 
     #3ª tabela EXAME
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = 'DB_Pacientes_Actual!S4', valueInputOption = 'USER_ENTERED', body = {'values': dfs[6].values.tolist()}).execute())
    #Apaga as linhas abaixo dos dados adicionados
    numero_linhas_dados = len(dfs[6])
    intervalo_apagar = f'DB_Pacientes_Actual!S{numero_linhas_dados + 100}:U'
    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=intervalo_apagar).execute() 
        
     #1ª tabela CONSULTA - ESPECIFICAR SEMANA
    df_filtrado = dfs3[1].loc[df.eq(semana).any(axis=1)]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = consultaPrimeiraTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute())
     #2ª tabela CONSULTA - ESPECIFICAR MÊS
    df_filtrado = dfs[3][dfs[3]['Month/Year (Data_EXECUCAO)'] == mesAno]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = consultaSegundaTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute()) 
     #3ª tabela CONSULTA
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = 'DB_Pacientes_Actual!AD4', valueInputOption = 'USER_ENTERED', body = {'values': dfs[4].values.tolist()}).execute())
    #Apaga as linhas abaixo dos dados adicionados
    numero_linhas_dados = len(dfs[4])
    intervalo_apagar = f'DB_Pacientes_Actual!AD{numero_linhas_dados + 100}:AF'
    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=intervalo_apagar).execute() 
        
     #1ª tabela ODONTO - ESPECIFICAR SEMANA
    df_filtrado = dfs3[3].loc[df.eq(semana).any(axis=1)]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = odontoPrimeiraTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute())
     #2ª tabela ODONTO - ESPECIFICAR MÊS
    df_filtrado = dfs[7][dfs[7]['Month/Year (Data_EXECUCAO)'] == mesAno]
    dados_para_atualizar = df_filtrado['COUNT_PAC_PAG'].tolist()
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = odontoSegundaTabela, valueInputOption = 'USER_ENTERED', body = {'values': [dados_para_atualizar]}).execute())
     #3ª tabela ODONTO
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = 'DB_Pacientes_Actual!AO4', valueInputOption = 'USER_ENTERED', body = {'values': dfs[8].values.tolist()}).execute())
    #Apaga as linhas abaixo dos dados adicionados
    numero_linhas_dados = len(dfs[8])
    intervalo_apagar = f'DB_Pacientes_Actual!AO{numero_linhas_dados + 100}:AQ'
    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=intervalo_apagar).execute()
    
    
     #Edição guia "DB_GSV_Clínica_Actual":
    dfs[1] = dfs[1].replace({np.nan: None})
    dfs[1] = dfs[1].drop(dfs[1].index[-1])
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = 'DB_GSV_Clínica_Actual!A2', valueInputOption = 'USER_ENTERED', body = {'values': dfs[1].values.tolist()}).execute())
    #Apaga as linhas abaixo dos dados adicionados
    numero_linhas_dados = len(dfs[1])
    intervalo_apagar = f'DB_GSV_Clínica_Actual!A{numero_linhas_dados + 100}:F'
    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=intervalo_apagar).execute()
    
     #Edição guia "DB_TME_Actual":
       
     #1ª tabela TME - ESPECIFICAR LINHA DO MÊS
    tme_senha = [round(num) for num in dfs2[0]['TME Senha'].iloc[-1:].tolist()]
    tme_recepcao = [round(num) for num in dfs2[0]['TMA Recepção'].iloc[-1:].tolist()]
    tme_consulta = [round(num) for num in dfs2[0]['TME Consulta'].iloc[-1:].tolist()]
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = TMEPrimeiraTabelaC1, valueInputOption = 'USER_ENTERED', body = {'values': [tme_senha]}).execute())
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = TMEPrimeiraTabelaC2, valueInputOption = 'USER_ENTERED', body = {'values': [tme_recepcao]}).execute())
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = TMEPrimeiraTabelaC3, valueInputOption = 'USER_ENTERED', body = {'values': [tme_consulta]}).execute())
     #2ª tabela TME - ESPECIFICAR LINHA DO DIA
    filtro_ultimas_linhas = [[round(num) for num in sublist] for sublist in dfs2[1][['TME Senha', 'TMA Recepção', 'TME Consulta']].dropna().iloc[-2:].values.tolist()]
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = TMESegundaTabela, valueInputOption = 'USER_ENTERED', body = {'values': filtro_ultimas_linhas}).execute())
     #3ª tabela TME - ESPECIFICAR LINHA DA SEMANA
    colunas_desejadas = [[round(num) for num in sublist] for sublist in dfs2[2][['TME Senha', 'TMA Recepção', 'TME Consulta']].dropna().iloc[-2:].values.tolist()]
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = TMETerceiraTabela, valueInputOption = 'USER_ENTERED', body = {'values': colunas_desejadas}).execute()) 

    
     #Edição guia "DB_consultas_Actual":
    dfs[10] = dfs[10].replace({np.nan: None})
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = 'DB_consultas_Actual!A2', valueInputOption = 'USER_ENTERED', body = {'values': dfs[10].values.tolist()}).execute())
    #Apaga as linhas abaixo dos dados adicionados
    numero_linhas_dados = len(dfs[10])
    intervalo_apagar = f'DB_consultas_Actual!A{numero_linhas_dados + 100}:E'
    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=intervalo_apagar).execute()
    
     #Edição guia "DB_avaliação_Actual":
    dfs[0] = dfs[0].replace({np.nan: None})
    result = (sheet.values().update(spreadsheetId = SAMPLE_SPREADSHEET_ID, range = 'DB_avaliação_Actual!A2', valueInputOption = 'USER_ENTERED', body = {'values': dfs[0].values.tolist()}).execute())
    #Apaga as linhas abaixo dos dados adicionados
    numero_linhas_dados = len(dfs[0])
    intervalo_apagar = f'DB_avaliação_Actual!A{numero_linhas_dados + 100}:F'
    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=intervalo_apagar).execute()

except HttpError as err:
    print(err)
    
print("Edição finalizada! Verificar Sheets...")